# Sobre o banco de dados

Conjunto de dados de cancelamentos e atrasos de voos de companhias aéreas de Janeiro de 2023, dados extraídos do Departamento de Transportes dos EUA, Bureau of Transportation Statistics (https://www.transstats.bts.gov) e disponível no Kaggle.

As variáveis ​​incluem rotas de voo (origem e destino), intervalo de tempo entre os eventos (minutos, tempo no destino), motivos/atribuições de atrasos e cancelamentos.

## Descrição das variáveis

### Tabela DOTCODEDICTIONARY

| Variável | Descrição |
| -------- | --------- |
| Code | Identificador numérico do U.S. Department of Transportation (DOT) para companhias aéreas |
| Description | Descrição da companhia aérea |

### Tabela AIRLINECODEDICTIONARY

| Variável | Descrição |
| -------- | --------- |
| Code | Código exclusivo do operador para agências operadoras de aeronaves |
| Description | Descrição do órgão operador da aeronave |

### Tabela voos_202301

| Variável | Descrição |
| -------- | --------- |
| FL_DATE | Data do voo (aaaammdd) |
| AIRLINE_CODE | Código exclusivo da operadora. Quando vários operadores utilizam o mesmo código, um sufixo numérico é usado para diferenciar usuários anteriores, por exemplo, PA, PA(1), PA(2). |
| DOT_CODE | Um número de identificação atribuído pelo US DOT para identificar uma companhia aérea única (transportadora). Uma companhia aérea única (transportadora) é definida como aquela que possui e reporta sob o mesmo certificado DOT independentemente do seu código, nome ou holding/sociedade. |
| FL_NUMBER | Número do voo |
| ORIGIN | Aeroporto de origem |
| ORIGIN_CITY | Aeroporto de origem, nome da cidade |
| DEST | aeroporto de destino |
| DEST_CITY | Aeroporto de destino, nome da cidade |
| CRSDEPTIME | Hora de partida registrada CRS (Sistema de Controle de Reservas) (horário local: hhmm) |
| DEP_TIME | Hora real de partida (hora local: hhmm) |
| DEP_DELAY | Diferença em minutos entre o horário de partida programado e o real. As saídas antecipadas apresentam números negativos. |
| TAXI_OUT | Tempo de táxi na saída em minutos (táxi é o processo de mover um avião enquanto ele está na pista) |
| WHEELS_OFF | Hora exata de decolagem (hora local: hhmm) |
| WHEELS_ON | Hora exata de pouso (hora local: hhmm) |
| TAXI_IN | Tempo de táxi na chegada em minutos |
| CRSARRTIME | Hora de chegada registrada no CRS (hora local: hhmm) |
| ARR_TIME | Hora real de chegada (hora local: hhmm) |
| ARR_DELAY | Diferença em minutos entre o horário previsto de chegada e o real. As chegadas antecipadas mostram números negativos. |
| CANCELLED | Indicador de voo cancelado (1=Sim) |
| CANCELLATION_CODE | Especifica o motivo do cancelamento |
| DIVERTED | Indicador de voo desviado (1=Sim) |
| CRSELAPSEDTIME | Tempo total de voo decorrido em minutos registrado no CRS |
| ELAPSED_TIME | Tempo total de voo decorrido em minutos reais |
| AIR_TIME | Tempo de voo no ar em minutos |
| DISTANCE | Distância entre aeroportos (milhas) |
| DELAYDUECARRIER | Atraso da operadora em minutos |
| DELAYDUEWEATHER | Atraso climático em minutos |
| DELAYDUENAS | Atraso do Sistema Aéreo Nacional em Minutos |
| DELAYDUESECURITY | Atraso de segurança em minutos |
| DELAYDUELATE_AIRCRAFT | Atraso de aeronaves atrasadas em minutos |

# Imports

In [1]:
import pandas as pd

In [3]:
df_DotCodeDictionary = pd.read_excel("../dataset/DOT_CODE_DICTIONARY.xlsx")
df_AirlineCodeDictionary = pd.read_csv("../dataset/AIRLINE_CODE_DICTIONARY.csv")
df_voos202301 = pd.read_csv("../dataset/flights_202301.csv")

In [4]:
df_DotCodeDictionary.head()

,Code,Description
0,19031,Mackey International Inc.: MAC
1,19032,Munz Northern Airlines Inc.: XY
2,19033,Cochise Airlines Inc.: COC
3,19034,Golden Gate Airlines Inc.: GSA
4,19035,Aeromech Inc.: RZZ


In [5]:
df_AirlineCodeDictionary.head()

,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


In [6]:
df_voos202301.head()

,FL_DATE,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,CRS_DEP_TIME,DEP_TIME,...,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT,FL_YEAR,FL_MONTH,FL_DAY
0,2023-01-02,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,757.0,...,25.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,2
1,2023-01-03,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,755.0,...,37.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,3
2,2023-01-04,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,755.0,...,28.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,4
3,2023-01-05,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,754.0,...,38.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,5
4,2023-01-06,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,759.0,...,28.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,6
